In [ ]:
import os
from fpdf import FPDF
import pandas as pd
import pickle
import shap
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

# Load model and vectorizer
with open("../models/toxicity_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("../models/vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

# Sample inputs to explain
sample_prompts = [
    "I hate you, go to hell.",
    "You're such a kind and wonderful person.",
    "Stupid idiot, learn to speak properly!",
    "Let's have a respectful conversation.",
    "You filthy scumbag!"
]

X = vectorizer.transform(sample_prompts)
preds = model.predict_proba(X)[:, 1]

# SHAP explanations
explainer = shap.Explainer(model, X)
shap_values = explainer(X)

# Save SHAP force plots
shap_folder = "shap_explanations"
os.makedirs(shap_folder, exist_ok=True)
force_plot_paths = []

for i, val in enumerate(sample_prompts):
    out_path = f"{shap_folder}/force_plot_{i}.png"
    shap.plots.force(shap_values[i], matplotlib=True, show=False)
    plt.savefig(out_path, bbox_inches='tight')
    plt.close()
    force_plot_paths.append(out_path)

# Collect all pre-existing plots
plot_dir = "../plots"
plot_files = [os.path.join(plot_dir, f) for f in os.listdir(plot_dir) if f.endswith(('.png', '.jpg'))]
plot_files.sort()

# Generate PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()

pdf.set_font("Arial", 'B', 16)
pdf.cell(0, 10, "BiasShield - Toxicity & Bias Detection Report", ln=True)

# Section 1: Sample Prompts + SHAP
for i, prompt in enumerate(sample_prompts):
    pdf.set_font("Arial", 'B', 12)
    pdf.ln(10)
    pdf.cell(0, 10, f"Prompt {i+1}:", ln=True)
    pdf.set_font("Arial", '', 11)
    pdf.multi_cell(0, 10, prompt)
    pdf.cell(0, 10, f"Predicted Toxicity Score: {round(preds[i]*100, 2)}%", ln=True)
    pdf.image(force_plot_paths[i], w=160)

# Section 2: Visual Plots
pdf.add_page()
pdf.set_font("Arial", 'B', 14)
pdf.cell(0, 10, "Exploratory Visualizations", ln=True)

for plot in plot_files:
    pdf.ln(8)
    pdf.image(plot, w=180)

# Save report
os.makedirs("pdf_reports", exist_ok=True)
pdf_path = "pdf_reports/biasshield_report.pdf"
pdf.output(pdf_path)

print(f"✅ PDF report created at: {pdf_path}")


/var/folders/cz/hz_5566j3_vcr8nvj_0mkh_80000gn/T/ipykernel_51305/493583542.py:54: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", 'B', 16)
/var/folders/cz/hz_5566j3_vcr8nvj_0mkh_80000gn/T/ipykernel_51305/493583542.py:55: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "BiasShield - Toxicity & Bias Detection Report", ln=True)
/var/folders/cz/hz_5566j3_vcr8nvj_0mkh_80000gn/T/ipykernel_51305/493583542.py:59: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", 'B', 12)
/var/folders/cz/hz_5566j3_vcr8nvj_0mkh_80000gn/T/ipykernel_51305/493583542.py:61: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f"Pr

FPDFUnicodeEncodingException: Character "📊" at index 0 in text is outside the range of characters supported by the font used: "helveticaB". Please consider using a Unicode font.